This model:

1. Loads adapter database and defines scoring weights
2. Calculates comprehensive compatibility scores based on:
   - Power delivery capabilities
   - Data transfer speeds
   - Display support
   - Cable quality
   - Additional features

3. Filters adapters based on user requirements:
   - Connector types
   - Required features
   - Minimum specifications

4. Provides recommendations with:
   - Compatibility scores
   - Category ratings
   - Feature breakdowns
   - Quality assessments

5. Supports customization through:
   - Adjustable scoring weights
   - Custom filtering rules
   - Feature importance settings
   - Category thresholds

To implement this in a user interface:
1. Create a form for user requirements
2. Process selections through the model
3. Display results with visualizations
4. Allow filtering and sorting of recommendations
5. Show detailed comparisons of selected adapters

This provides a systematic way to match user needs with compatible adapters while considering all relevant features and requirements.


In [ ]:
import pandas as pd
import numpy as np

class AdapterSelector:
    def __init__(self, adapter_database):
        self.db = pd.read_csv(adapter_database)
        self.weights = {
            'power_delivery': 20,
            'data_transfer': 20,
            'display_support': 15,
            'cable_quality': 10,
            'additional_features': 20,
            'compatibility': 15
        }
    
    def calculate_score(self, adapter):
        """Calculate compatibility score based on features"""
        score = 0
        
        # Power Delivery Score
        if adapter['supports_usb_pd']:
            pd_watts = float(adapter['max_power_delivery'].split('V')[0]) * \
                      float(adapter['max_power_delivery'].split('/')[1].split('A')[0])
            score += min(pd_watts/100 * self.weights['power_delivery'], self.weights['power_delivery'])
        
        # Data Transfer Score
        data_rates = {
            'USB4': 20, 'USB 3.2': 15, 'USB 3.1': 10,
            'USB 3.0': 5, 'USB 2.0': 3, 'USB1.1': 1
        }
        score += data_rates.get(adapter['usb_version'], 0)
        
        # Display Support Score
        display_score = 0
        if adapter['supports_displayport']: display_score += 5
        if adapter['supports_hdmi']: display_score += 5
        if adapter['supports_alt_mode']: display_score += 5
        score += min(display_score, self.weights['display_support'])
        
        # Cable Quality Score
        wire_gauge = int(adapter['wire_gauge'].split()[0])
        quality_score = (30 - wire_gauge) / 6 * self.weights['cable_quality']
        if adapter['shielding_type'] == 'Foil+Braid':
            quality_score = min(quality_score + 5, self.weights['cable_quality'])
        score += quality_score
        
        # Additional Features Score
        feature_score = 0
        if adapter['supports_thunderbolt']: feature_score += 10
        if adapter['Qualcomm Quick Charge 4+']: feature_score += 5
        if adapter['supports_audio']: feature_score += 5
        score += min(feature_score, self.weights['additional_features'])
        
        return score

    def filter_adapters(self, requirements):
        """Filter adapters based on user requirements"""
        filtered_db = self.db.copy()
        
        # Basic filtering
        if requirements.get('connector_type_a'):
            filtered_db = filtered_db[
                filtered_db['connector_type_a'] == requirements['connector_type_a']
            ]
        if requirements.get('connector_type_b'):
            filtered_db = filtered_db[
                filtered_db['connector_type_b'] == requirements['connector_type_b']
            ]
            
        # Feature requirements
        if requirements.get('needs_power_delivery'):
            filtered_db = filtered_db[filtered_db['supports_usb_pd'] == True]
        if requirements.get('needs_display'):
            filtered_db = filtered_db[
                (filtered_db['supports_displayport'] == True) | 
                (filtered_db['supports_hdmi'] == True)
            ]
            
        return filtered_db

    def get_recommendations(self, user_requirements):
        """Get adapter recommendations based on user requirements"""
        filtered_adapters = self.filter_adapters(user_requirements)
        
        # Calculate scores for filtered adapters
        filtered_adapters['compatibility_score'] = filtered_adapters.apply(
            self.calculate_score, axis=1
        )
        
        # Sort by score and categorize
        filtered_adapters['category'] = pd.cut(
            filtered_adapters['compatibility_score'],
            bins=[0, 59, 69, 79, 89, 100],
            labels=['Basic', 'Standard', 'Advanced', 'Premium', 'Ultimate']
        )
        
        return filtered_adapters.sort_values('compatibility_score', ascending=False)


FileNotFoundError: [Errno 2] No such file or directory: 'Adapters-Type-C.csv'

In [ ]:

# Example usage:
user_requirements = {
    'connector_type_a': 'USB-C',
    'connector_type_b': 'USB-A',
    'needs_power_delivery': True,
    'needs_display': True
}


In [ ]:

selector = AdapterSelector('Adapters-Type-C.csv')
recommendations = selector.get_recommendations(user_requirements)